# Yield analysis of umux100K

Finds channels that do not pass the following criteria
1. in smurf keepout zone
2. frequency collisions
3. no squid modulation
4. squid lambda (weird v-phi curves)

## Notes
 - using 'device_summary.csv' provided by Caleb on 5/21/2021, linked in nist/pton friday meeting 
 - All microwave powers are -20dBm, which is ~-77dBm on feedline
 - duplicate rows for wafer 11,13?
 - dfpp is a string due to "None" entries
 
 - wafer 9: good 
 - wafer 11: duplicate row entries.  Band (0,2,11,13) have (2,2,3,3) copies of equivalent data sets.  Thus need to slice to particular row indices.
   - upper sub-band of Band0 has resonant structure in v-phi
 - wafer 12: like wafer 11 has duplicate row entries Band (0,2,11,13) have (2,2,3,3) copies of equivalent data sets.
 - wafer 13: punt on lambda modulation yield due to incorrectly finding weird phis
   - all fits are for higher rf port power (-20dB) and therefore lambda fits are meaningless.  
   - Need -40dB lambda fits in order to use chi_squared to get "weird phi" 
   - band00, all channels have dfpp close (but not exactly) = 0. This must be incorrect since there are non-zero lambda values for band00. Looking at curves directly, the data is garbage.  Need to throw out this from yield analysis.  Res # 74 starts with good data. 
   - highly variable chi_squared values.  Need to look at curves directly.
   - Looking at curves directly for "weird vphis": 
     - band 01 upper subband res_num 34 - 48 all have. 
     - band 03 not really
     - band 05 lots 
     - band 07 none
     - band 09 none
     - band 11 none
     - band 13 none
     - band 00, unknown, bad data
     - band 02, none
     - band 04, yes for lower subband
     - band 06, no
     - band 08, no ??
     - band 10, upper sub-band a few channels
     - band 12, none
     
 
   
## To do
 - chip to chip collisions?


In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

/Users/hubmayr/.pyenv/versions/3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pandas.read_csv('device_summary.csv')

In [3]:
# some settings
#pandas.set_option('display.max_colwidth', -1)
#pandas.set_option("max_columns", None)
#pandas.set_option("max_rows", 20) # putting None will crush the notebook 


In [4]:
df.head()

,record_id,device_id,group_id,wafer_and_chip_id,lambda_path,f_ghz,so_band,is_in_band,is_in_keepout,res_num,...,left_neighbor_within_600.0_khz,left_neighbor_within_700.0_khz,left_neighbor_within_800.0_khz,left_neighbor_within_900.0_khz,right_neighbor_within_1000.0_khz,right_neighbor_within_500.0_khz,right_neighbor_within_600.0_khz,right_neighbor_within_700.0_khz,right_neighbor_within_800.0_khz,right_neighbor_within_900.0_khz
0,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01|Res0000,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01,/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.165673,1,True,False,0,...,False,False,False,False,False,False,False,False,False,False
1,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01|Res0001,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01,/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.168882,1,True,False,1,...,False,False,False,False,False,False,False,False,False,False
2,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01|Res0002,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01,/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.171067,1,True,False,2,...,False,False,False,False,True,False,False,True,True,True
3,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01|Res0003,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01,/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.171757,1,True,False,3,...,False,True,True,True,False,False,False,False,False,False
4,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01|Res0004,Wafer9|Band01|scan3.800GHz-6.200GHz_2021-01-26...,Wafer9|Band01,/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.173317,1,True,False,4,...,False,False,False,False,False,False,False,False,False,False


In [5]:
df.columns

Index(['record_id', 'device_id', 'group_id', 'wafer_and_chip_id',
       'lambda_path', 'f_ghz', 'so_band', 'is_in_band', 'is_in_keepout',
       'res_num', 'designed_f_ghz', 'x_pos_mm_on_chip', 'y_pos_mm_on_chip',
       'resonator_height_um', 'wiggles', 'sliders', 'slider_delta_um',
       'resonator_impedance_ohms', 'coupling_capacitance_f',
       'coupling_inductance_h', 'group_num', 'flags', 'wafer', 'chip_id',
       'lamb', 'lamb_err', 'flux_ramp_pp_khz', 'flux_ramp_pp_khz_err',
       'fr_squid_mi_pH', 'fr_squid_mi_pH_err', 'chi_squared', 'q_i_mean',
       'q_i_std', 'q_c_mean', 'q_c_std', 'impedance_ratio_mean',
       'left_neighbor_within_1000.0_khz', 'left_neighbor_within_500.0_khz',
       'left_neighbor_within_600.0_khz', 'left_neighbor_within_700.0_khz',
       'left_neighbor_within_800.0_khz', 'left_neighbor_within_900.0_khz',
       'right_neighbor_within_1000.0_khz', 'right_neighbor_within_500.0_khz',
       'right_neighbor_within_600.0_khz', 'right_neighbor_within_

In [6]:
N = len(df) # total number of rows
print(N)

7427


In [7]:
# Are there multiple port powers used?  If yes then there are multiple rows per unique resonator
port_power_dbm=[] 
for ii in range(N):
    port_power_dbm.append(int(df['record_id'][ii].split('|')[3]))
set(port_power_dbm)

{-20, -10, 0}

In [8]:
# add column that is the the port power to make life easier
df['port_power_dbm'] = port_power_dbm
df.columns

Index(['record_id', 'device_id', 'group_id', 'wafer_and_chip_id',
       'lambda_path', 'f_ghz', 'so_band', 'is_in_band', 'is_in_keepout',
       'res_num', 'designed_f_ghz', 'x_pos_mm_on_chip', 'y_pos_mm_on_chip',
       'resonator_height_um', 'wiggles', 'sliders', 'slider_delta_um',
       'resonator_impedance_ohms', 'coupling_capacitance_f',
       'coupling_inductance_h', 'group_num', 'flags', 'wafer', 'chip_id',
       'lamb', 'lamb_err', 'flux_ramp_pp_khz', 'flux_ramp_pp_khz_err',
       'fr_squid_mi_pH', 'fr_squid_mi_pH_err', 'chi_squared', 'q_i_mean',
       'q_i_std', 'q_c_mean', 'q_c_std', 'impedance_ratio_mean',
       'left_neighbor_within_1000.0_khz', 'left_neighbor_within_500.0_khz',
       'left_neighbor_within_600.0_khz', 'left_neighbor_within_700.0_khz',
       'left_neighbor_within_800.0_khz', 'left_neighbor_within_900.0_khz',
       'right_neighbor_within_1000.0_khz', 'right_neighbor_within_500.0_khz',
       'right_neighbor_within_600.0_khz', 'right_neighbor_within_

In [9]:
# which wafers are in the file?
wafers = set(df["wafer"])
print(wafers)

{9, 11, 12, 13, 14}


In [10]:
# convert flux_ramp_pp_khz/err, lamb/err, and chi_squared for entire data frame to numeric values rather than strings
# None replaced with Nan to enable plotting
cols = ['flux_ramp_pp_khz','flux_ramp_pp_khz_err','lamb','lamb_err','chi_squared']
for field in cols:
    tmp_arr = pandas.to_numeric(df[field], errors='coerce')
    df[field]=tmp_arr

#### some helper functions

In [11]:
def get_chipID_for_wafer_and_band(wafer,band):
    df_wafer = df[df["wafer"] == wafer]
    df_wafer_band = df_wafer[df_wafer["so_band"] == band]
    chip_ids = list(set(df_wafer_band["chip_id"]))
    return chip_ids

def get_rf_powers_for_wafer_and_chip(wafer,chip_id):
    df_wafer = df[df["wafer"] == wafer]
    port_powers = list(set(df_wafer[df_wafer['chip_id'] == chip_id]['port_power_dbm']))
    return port_powers
#get_rf_powers_for_wafer_and_chip(11,"Band00")

# Yield analysis on single wafer defined here

In [12]:
wafer_num = 13

## Determine bands, chips, microwave powers, and if duplicate rows for this wafer

In [13]:
df_wafer = df[df["wafer"] == wafer_num] # slice to only one wafer
bands = sorted(list(set(df_wafer["so_band"])))
chips = sorted(list(set(df_wafer["chip_id"])))
N_chips = len(chips)
N_res_expect = 0 
for band in list(bands):
    chips_for_band = set(df_wafer[df_wafer["so_band"] == band]["chip_id"])
    N_chips_per_band = len(chips_for_band)
    print(N_chips_per_band, " chip(s) for Band ",band,". chips: ", chips_for_band)
    N_res_expect = N_res_expect + N_chips_per_band*66
print('Expect %d resonators'%N_res_expect)


1  chip(s) for Band  0 . chips:  {'Band00_(-1.000&4.000)'}
1  chip(s) for Band  1 . chips:  {'Band01_(-1.000&3.000)'}
1  chip(s) for Band  2 . chips:  {'Band02_(-1.000&-4.000)'}
1  chip(s) for Band  3 . chips:  {'Band03_(1.000&-1.000)'}
1  chip(s) for Band  4 . chips:  {'Band04_(1.000&0.000)'}
1  chip(s) for Band  5 . chips:  {'Band05_(-1.000&-1.000)'}
1  chip(s) for Band  6 . chips:  {'Band06_(-1.000&-2.000)'}
1  chip(s) for Band  7 . chips:  {'Band07_(0.000&-1.000)'}
1  chip(s) for Band  8 . chips:  {'Band08_(0.000&-2.000)'}
1  chip(s) for Band  9 . chips:  {'Band09_(0.000&-3.000)'}
1  chip(s) for Band  10 . chips:  {'Band10_(0.000&4.000)'}
1  chip(s) for Band  11 . chips:  {'Band11_(0.000&-5.000)'}
1  chip(s) for Band  12 . chips:  {'Band12_(0.000&-6.000)'}
1  chip(s) for Band  13 . chips:  {'Band13_(0.000&-7.000)'}
Expect 924 resonators


In [14]:
for chip in chips:
    print(chip,' port powers: ',get_rf_powers_for_wafer_and_chip(wafer_num,chip)) 

Band00_(-1.000&4.000)  port powers:  [-20]
Band01_(-1.000&3.000)  port powers:  [-20]
Band02_(-1.000&-4.000)  port powers:  [-20]
Band03_(1.000&-1.000)  port powers:  [-20]
Band04_(1.000&0.000)  port powers:  [-20]
Band05_(-1.000&-1.000)  port powers:  [-20]
Band06_(-1.000&-2.000)  port powers:  [-20]
Band07_(0.000&-1.000)  port powers:  [-20]
Band08_(0.000&-2.000)  port powers:  [-20]
Band09_(0.000&-3.000)  port powers:  [-20]
Band10_(0.000&4.000)  port powers:  [-20]
Band11_(0.000&-5.000)  port powers:  [-20]
Band12_(0.000&-6.000)  port powers:  [-20]
Band13_(0.000&-7.000)  port powers:  [-20]


In [15]:
# # plot frequency versus dataframe index; easy way to tell if there are multiple rows per resonator
# for ii, chip in enumerate(chips):
#     df_chip = df_wafer[df_wafer['chip_id'] == chip]
#     plt.figure(ii)
#     plt.plot(df_chip.index,df_chip['f_ghz']*1000,'o-')
#     plt.xlabel('dataframe index')
#     plt.ylabel('F (MHz)')
#     plt.title('Wafer %d %s'%(wafer_num,chip))
# plt.show()

In [16]:
# compare two rows for a given res_num to see if duplicative
df_chip = df_wafer[df_wafer["chip_id"] == "Band11"]
df_chip[df_chip['res_num']==0]

,record_id,device_id,group_id,wafer_and_chip_id,lambda_path,f_ghz,so_band,is_in_band,is_in_keepout,res_num,...,left_neighbor_within_700.0_khz,left_neighbor_within_800.0_khz,left_neighbor_within_900.0_khz,right_neighbor_within_1000.0_khz,right_neighbor_within_500.0_khz,right_neighbor_within_600.0_khz,right_neighbor_within_700.0_khz,right_neighbor_within_800.0_khz,right_neighbor_within_900.0_khz,port_power_dbm


In [17]:
# Plot the different duplicate row entries to see if results are the same or different.
chip_msets = {}
for ii,chip in enumerate(chips):
    df_chip = df_wafer[df_wafer["chip_id"]==chip]
    start_ind = df_chip[df_chip["res_num"]==0].index.to_list()
    end_ind = df_chip[df_chip["res_num"]==np.max(df_chip["res_num"].to_numpy())].index.to_list()
    print(start_ind,end_ind)
    N = len(start_ind)
    plt.figure(20+ii)
    plt.title(chip)
    plt.xlabel('resonator index')
    plt.ylabel('Frequency MHz')
    df_msets=[]
    for jj in range(N):
        df_mset = df_chip.loc[start_ind[jj]:end_ind[jj]]
        df_msets.append(df_mset)
        plt.plot(df_mset["res_num"].to_numpy(),df_mset['f_ghz'].to_numpy()*1000,'o-',label=jj)
    plt.legend()
    chip_msets[chip] = df_msets
plt.show()
    
    #print(start_ind,end_ind)

[1768, 1835] [1834, 1901]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[1902, 1967] [1966, 2031]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[2032, 2098] [2097, 2163]


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[3316, 3383] [3382, 3449]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[3450, 3516] [3515, 3581]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Ok, since there is duplicate data, need to make a data slice "df_wafer" that has only one data set per chip

In [18]:
dexes =[]
for chip in chips:
    dexes.extend(chip_msets[chip][0].index.to_list())
df_wafer = df.loc[dexes]

# outside smurf keepout zone

In [19]:
cut_smurfkeepout_indices = df_wafer.index[df_wafer['is_in_keepout']].to_list()

# Frequency Collisions

In [20]:
def get_collision_report(f_arr_ghz,df_collision_MHz=0.5,wafer_number=0,fig_num=1):
    ''' report statistics and plot histogram of nearest frequency neighbors.  
        f_arr_MHz must be in ascending order.
        
        returns the frequenices which are collided
    '''
    f = f_arr_ghz 
    f_diff = np.diff(f)*1000 # to MHz
    N = len(f)
    
    # cut out very large frequency differences, which throw off the distribution statistics
    df_cull_indices = np.where(f_diff < 10)[0]
    df_cull = f_diff[df_cull_indices]
    N_cut = len(f_diff) - len(df_cull)
    dfs_cut = f_diff[np.where(f_diff > 10)[0]]
    print(N, "resonators.  ", N_cut, " f_diffs cut with values: ",dfs_cut)
    
    s = pandas.Series(df_cull)
    print(s.describe()) # high level statistic 
    
    collision_indices = np.where(f_diff <= df_collision_MHz)[0]
    bad_fs_ghz = []
    for dex in collision_indices:
        bad_fs_ghz.extend(f[dex:dex+2])
    bad_fs_ghz = sorted(list(set(bad_fs_ghz)))
    N_collided_res = len(bad_fs_ghz)
    res_yield_percent = 100-N_collided_res/N*100
    print(N_collided_res, 'collided resonators in ',N, ' resonators\nYield = %.1f'%(res_yield_percent))
    
    plt.figure(fig_num)
    plt.hist(f_diff,bins=100,range=(0,10),density=True) # 100 kHz bins, the target BW
    plt.title('Wafer %d Frequency collisions'%wafer_number)
    #plt.xlim(0,10)
    plt.axvline(x=.3,color='r')
    plt.xlabel('$\Delta$f (MHz)')
    plt.ylabel('Density')
    plt.text(6,0.5, "N_res = %d\nN_collided = %d\nYield = %.1f%%"%(N,N_collided_res,res_yield_percent))
    plt.show()
    
    return bad_fs_ghz

In [21]:
# # for wafers (like 14) with duplicate chips per band
# group0_chips = []
# for ii in range(0,28,2):
#     group0_chips.append(chips[ii])
# group1_chips = []
# for ii in range(1,28,2):
#     group1_chips.append(chips[ii])

# for wafers without duplicate chips per band, do this    
f_arr_ghz = sorted(df_wafer["f_ghz"].to_numpy())

#df_group = df_wafer_power[df_wafer_power["chip_id_str"].isin(group1_chips)]    
#f_arr_MHz = df_group["f_ghz"].to_numpy()*1000

In [22]:
collided_fs = get_collision_report(f_arr_ghz,wafer_number=wafer_num,fig_num=100)

907 resonators.   7  f_diffs cut with values:  [40.71164917 11.6642305  11.32940564 31.93970902 11.18381929 13.11949976
 42.51480354]
count    899.000000
mean       2.054164
std        1.107353
min        0.000052
25%        1.590311
50%        1.930562
75%        2.301887
max        9.830699
dtype: float64
55 collided resonators in  907  resonators
Yield = 93.9


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
# get dataframe indices for collided resonators.  How to do this w/out a for loop?
cut_f_indices = []
for f in collided_fs:
    cut_f_indices.extend(df_wafer.index[df_wafer['f_ghz'] == f].to_list())
#cut_f_indices = sorted(list(set(cut_f_indices)))
df.loc[cut_f_indices]
    
    

,record_id,device_id,group_id,wafer_and_chip_id,lambda_path,f_ghz,so_band,is_in_band,is_in_keepout,res_num,...,left_neighbor_within_700.0_khz,left_neighbor_within_800.0_khz,left_neighbor_within_900.0_khz,right_neighbor_within_1000.0_khz,right_neighbor_within_500.0_khz,right_neighbor_within_600.0_khz,right_neighbor_within_700.0_khz,right_neighbor_within_800.0_khz,right_neighbor_within_900.0_khz,port_power_dbm
1829,Wafer13|Band00_(-1.000&4.000)|scan4.000GHz-6.1...,Wafer13|Band00_(-1.000&4.000)|Res0061,Wafer13|Band00_(-1.000&4.000)|scan4.000GHz-6.1...,Wafer13|Band00_(-1.000&4.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.173883,0,False,False,61,...,False,False,False,True,True,True,True,True,True,-20
1830,Wafer13|Band00_(-1.000&4.000)|scan4.000GHz-6.1...,Wafer13|Band00_(-1.000&4.000)|Res0062,Wafer13|Band00_(-1.000&4.000)|scan4.000GHz-6.1...,Wafer13|Band00_(-1.000&4.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.173884,0,False,False,62,...,True,True,True,False,False,False,False,False,False,-20
1904,Wafer13|Band01_(-1.000&3.000)|scan4.000GHz-6.1...,Wafer13|Band01_(-1.000&3.000)|Res0002,Wafer13|Band01_(-1.000&3.000)|scan4.000GHz-6.1...,Wafer13|Band01_(-1.000&3.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.185362,1,True,False,2,...,False,False,False,True,True,True,True,True,True,-20
1905,Wafer13|Band01_(-1.000&3.000)|scan4.000GHz-6.1...,Wafer13|Band01_(-1.000&3.000)|Res0003,Wafer13|Band01_(-1.000&3.000)|scan4.000GHz-6.1...,Wafer13|Band01_(-1.000&3.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.185387,1,True,False,3,...,True,True,True,False,False,False,False,False,False,-20
2164,Wafer13|Band03_(1.000&-1.000)|scan4.000GHz-6.1...,Wafer13|Band03_(1.000&-1.000)|Res0000,Wafer13|Band03_(1.000&-1.000)|scan4.000GHz-6.1...,Wafer13|Band03_(1.000&-1.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.447868,3,True,False,0,...,False,False,False,False,False,False,False,False,False,-20
2094,Wafer13|Band02_(-1.000&-4.000)|scan4.000GHz-6....,Wafer13|Band02_(-1.000&-4.000)|Res0062,Wafer13|Band02_(-1.000&-4.000)|scan4.000GHz-6....,Wafer13|Band02_(-1.000&-4.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.448259,2,False,False,62,...,False,False,False,False,False,False,False,False,False,-20
2097,Wafer13|Band02_(-1.000&-4.000)|scan4.000GHz-6....,Wafer13|Band02_(-1.000&-4.000)|Res0065,Wafer13|Band02_(-1.000&-4.000)|scan4.000GHz-6....,Wafer13|Band02_(-1.000&-4.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.453873,2,False,False,65,...,False,False,False,False,False,False,False,False,False,-20
2165,Wafer13|Band03_(1.000&-1.000)|scan4.000GHz-6.1...,Wafer13|Band03_(1.000&-1.000)|Res0001,Wafer13|Band03_(1.000&-1.000)|scan4.000GHz-6.1...,Wafer13|Band03_(1.000&-1.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.453900,3,True,False,1,...,False,False,False,True,False,False,False,False,False,-20
2166,Wafer13|Band03_(1.000&-1.000)|scan4.000GHz-6.1...,Wafer13|Band03_(1.000&-1.000)|Res0002,Wafer13|Band03_(1.000&-1.000)|scan4.000GHz-6.1...,Wafer13|Band03_(1.000&-1.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.454858,3,True,False,2,...,False,False,False,True,True,True,True,True,True,-20
2167,Wafer13|Band03_(1.000&-1.000)|scan4.000GHz-6.1...,Wafer13|Band03_(1.000&-1.000)|Res0003,Wafer13|Band03_(1.000&-1.000)|scan4.000GHz-6.1...,Wafer13|Band03_(1.000&-1.000),/Users/cwheeler/PycharmProjects/WaferScreen/wa...,4.454858,3,True,False,3,...,True,True,True,True,False,False,False,False,False,-20


# Flux modulation
  - Note that the values of vector "flux_ramp_pp_khz" as read in by pandas are of type str due to the possibiity of "None" in the field.  pandas lists it as an "object"
  - Wafer 9, index 34 and 441 has negative df_pp

### info on uncoupled resonators

In [24]:
df_wafer[df_wafer["res_num"] == 0][['so_band','res_num','f_ghz','flux_ramp_pp_khz','flux_ramp_pp_khz_err']]

,so_band,res_num,f_ghz,flux_ramp_pp_khz,flux_ramp_pp_khz_err
1768,0,0,4.044246,0.016094,1.554248e-11
1902,1,0,4.177557,0.090642,8.522493e-11
2032,2,0,4.315727,0.033867,4.507110e-11
2164,3,0,4.447868,0.080420,5.778907e-11
2292,4,0,4.610827,5.125708,7.015841e-07
2422,5,0,4.746453,0.053785,8.738813e-11
2554,6,0,4.885447,0.285633,6.852185e-10
2682,7,0,5.054310,0.110951,6.246010e-10
2794,8,0,5.188422,0.094373,1.122350e-10
2924,9,0,5.322724,0.095833,2.295005e-10


In [25]:
#df_wafer['flux_ramp_pp_khz'].astype('float64').describe()
def get_mod_report(df_slice,wafer_num,dfpp_min_khz=30.0,dfpp_max_khz=200.0,fig_num=1,remove_squidless=True):
    
    # remove resonator only channel
    if remove_squidless:
        print("SQUIDless resonator channel properties:")
        print(df_slice[df_slice["res_num"] == 0][['so_band','res_num','f_ghz','flux_ramp_pp_khz','flux_ramp_pp_khz_err']])  
        df_local = df_slice[df_slice["res_num"] != 0] # works if properly identified 
    else:
        df_local = df_slice
    
    N_tot = len(df_local)
    
    plt.figure(fig_num)
    plt.title('dfpp versus index')
    #plt.errorbar(range(N_tot),df_slice['flux_ramp_pp_khz'],yerr=df_slice['flux_ramp_pp_khz_err'])
    plt.plot(df_local.index,df_local['flux_ramp_pp_khz'].to_numpy(),'*-')
    plt.xlabel('dataframe index')
    plt.ylabel('df_pp (kHz)')

    plt.figure(fig_num+1)
    plt.title('dfpp error versus index')
    plt.plot(df_local.index, df_local['flux_ramp_pp_khz_err'].to_numpy(),'bo-')
    plt.xlabel('dataframe index')
    plt.ylabel('df_pp err (kHz)')

    plt.figure(fig_num+2)
    plt.title('Wafer %d SQUID modulation'%wafer_num)
    plt.hist(df_local['flux_ramp_pp_khz'],bins=50,range=(0,200),density=False)
    plt.xlabel('dfpp (kHz)')
    plt.ylabel('counts')
    plt.show()

    # yield cuts. Categories: null, dfpp low, dfpp high, dfpp_err = inf
    condition_labels = ['null','dfpp low','dfpp high', 'dfpp_err inf']
    conditions = [df_local['flux_ramp_pp_khz'].isnull(), 
                  df_local['flux_ramp_pp_khz']<dfpp_min_khz,
                  df_local['flux_ramp_pp_khz']>dfpp_max_khz,
                  df_local['flux_ramp_pp_khz_err']==np.inf
                 ]
    bad_indices_cat = [] # list of list per category
    bad_indices = [] # all categories combined
    for condition in conditions:
        tmp_list = df_local[condition].index.to_list()
        bad_indices_cat.append(tmp_list)
        bad_indices.extend(tmp_list)
    
    # produce good/bad indices
    bad_indices = sorted(list(set(bad_indices))) # remove duplicates
    good_indices = df_local.index.to_list()
    for dex in bad_indices:
        good_indices.remove(dex)
        
    N_good = len(good_indices)
    N_bad = len(bad_indices)
    yield_pct = N_good/N_tot*100
    
    
    print('\nStatistics on all resonators (squidless resonators removed = %s):'%remove_squidless)
    print(df_local['flux_ramp_pp_khz'].astype('float64').describe())
    
    print('\nStatistics on channels which passed cuts (squidless resonators removed = %s):'%remove_squidless)
    print(df_local.loc[good_indices,['flux_ramp_pp_khz']].describe())
                       
    print('\nSummary of channels cut:\n')
    print(df_local.loc[bad_indices,['chip_id','so_band','f_ghz','flux_ramp_pp_khz','flux_ramp_pp_khz_err','flags']])
    
    print('\n(N_tot,N_good,N_cut) = (%d,%d,%d)'%(N_tot,N_good,N_bad))
    print('Yield = ',yield_pct)
    
    print('Number cut by category')
    for ii, condition in enumerate(condition_labels):
        print(condition, ': ',(len(bad_indices_cat[ii])))
    
    return df_local, good_indices, bad_indices
    

    
        


In [26]:
df_local, good_fr_indices, cut_fluxramp_indices = get_mod_report(df_wafer,wafer_num,fig_num=200)

SQUIDless resonator channel properties:
      so_band  res_num     f_ghz  flux_ramp_pp_khz  flux_ramp_pp_khz_err
1768        0        0  4.044246          0.016094          1.554248e-11
1902        1        0  4.177557          0.090642          8.522493e-11
2032        2        0  4.315727          0.033867          4.507110e-11
2164        3        0  4.447868          0.080420          5.778907e-11
2292        4        0  4.610827          5.125708          7.015841e-07
2422        5        0  4.746453          0.053785          8.738813e-11
2554        6        0  4.885447          0.285633          6.852185e-10
2682        7        0  5.054310          0.110951          6.246010e-10
2794        8        0  5.188422          0.094373          1.122350e-10
2924        9        0  5.322724          0.095833          2.295005e-10
3054       10        0  5.439843         29.222031          3.295141e-09
3184       11        0  5.628061          0.050657          1.208356e-10
3316       

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


Statistics on all resonators (squidless resonators removed = True):
count     893.000000
mean      107.284189
std        92.727234
min        -4.758304
25%        81.075417
50%       107.832398
75%       132.680096
max      1768.432197
Name: flux_ramp_pp_khz, dtype: float64

Statistics on channels which passed cuts (squidless resonators removed = True):
       flux_ramp_pp_khz
count        797.000000
mean         111.637868
std           30.406501
min           33.646721
25%           88.757097
50%          112.498587
75%          134.665761
max          196.590742

Summary of channels cut:

                    chip_id  so_band     f_ghz  flux_ramp_pp_khz  \
1769  Band00_(-1.000&4.000)        0  4.050260          0.028029   
1770  Band00_(-1.000&4.000)        0  4.053086          0.050260   
1771  Band00_(-1.000&4.000)        0  4.053944          0.000396   
1772  Band00_(-1.000&4.000)        0  4.055943          0.021027   
1773  Band00_(-1.000&4.000)        0  4.057722          0.05

## Lambda Cuts
 - same as flux_ramp_khz_pp, need to convert since None and inf is an entry for some rows
 - using chi_squared as a proxy to find "weird" v-phis.  What num_chisquared_sigma is appropriate? 

In [27]:
def get_lambda_report(df_wafer,lamb_low_thresh=0, lamb_high_thresh=.6, num_chisquared_sigma=5,fig_num=300):
    ''' plot lambda report and return indices that do not pass the lambda cuts
    '''
    print(df_wafer[df_wafer["lamb_err"] != np.inf][['lamb','lamb_err','chi_squared']].describe())
    
    low_indices = df_wafer[df_wafer["lamb"] <= lamb_low_thresh].index.to_list()
    high_indices = df_wafer[df_wafer["lamb"] >= lamb_high_thresh].index.to_list()
    lamb_err_indices = df_wafer[df_wafer["lamb_err"] == np.inf].index.to_list()
    chi_s_mean = df_wafer["chi_squared"].to_numpy().mean()
    chi_s_std = df_wafer["chi_squared"].to_numpy().std()
    chis_indices = df_wafer[df_wafer["chi_squared"] > chi_s_mean+num_chisquared_sigma*chi_s_std].index.to_list()
    #chis_indices = df_wafer[df_wafer["chi_squared"] > 2e-11].index.to_list()
    
    cut_indices = []
    cut_lists = [low_indices,high_indices,lamb_err_indices,chis_indices]
    cut_list_labels=['low','high','lamb_err_inf','high chi squared']
    for tmp_list in cut_lists:
        cut_indices.extend(tmp_list)
    cut_indices = list(set(cut_indices))
    
    N = len(df_wafer)
    N_cut = len(cut_indices)
    yield_pct = (N-N_cut)/N*100
    
    print(N_cut, 'cut out of ', N, ' total channels.  Yield = %.1f%%'%(yield_pct))
    print("Cuts per category:")
    for ii,label in enumerate(cut_list_labels):
        print("%s: %d"%(label,len(cut_lists[ii])))
    
    plt.figure(fig_num)
    plt.errorbar(df_wafer.index.to_list(),df_wafer['lamb'],yerr=df_wafer['lamb_err'])
    plt.xlabel('index')
    plt.ylabel('lambda')
    plt.title("Wafer %d $\lambda$ fits"%(wafer_num))


    plt.figure(fig_num+1)
    plt.plot(df_wafer.index.to_list(),df_wafer['lamb_err'],'o')
    plt.xlabel('index')
    plt.ylabel('lambda')
    plt.title("Wafer %d $\lambda$ error"%(wafer_num))

    plt.figure(fig_num+2)
    plt.plot(df_wafer.index.to_list(),df_wafer['chi_squared'],'o')
    #plt.plot(df_wafer[df_wafer['chi_squared']<1].index.to_list(),df_wafer[df_wafer['chi_squared']<1]['chi_squared'],'o')
    plt.xlabel('index')
    plt.ylabel('lambda fit chi_squared')
    plt.ylim((0,1e-10))
    plt.title("Wafer %d chi_squared"%(wafer_num))
    plt.show()

    return cut_indices, low_indices, high_indices, lamb_err_indices, chis_indices
    
    

In [28]:
df_wafer_squidchannels = df_wafer[df_wafer["res_num"] != 0] # works if properly identified 
cut_lamb_indices, low_indices, high_indices, lamb_err_indices, chis_indices = get_lambda_report(df_wafer_squidchannels,fig_num=300)

             lamb      lamb_err   chi_squared
count  885.000000  8.850000e+02  8.850000e+02
mean     0.161913  4.137466e-02  1.472525e-05
std      0.155061  1.181941e+00  4.380073e-04
min     -0.910321  2.601223e-07  4.816379e-15
25%      0.075318  1.390517e-05  7.632957e-12
50%      0.167464  2.665868e-05  1.612510e-11
75%      0.248925  8.663310e-05  2.962758e-11
max      1.116506  3.516149e+01  1.303026e-02
111 cut out of  893  total channels.  Yield = 87.6%
Cuts per category:
low: 94
high: 11
lamb_err_inf: 8
high chi squared: 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-27-b906d7a0e309>:43: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(fig_num+2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

High chi_squared on wafer 13.  Need to look at curves directly.  

# Summarize all cuts

In [29]:
cut_dict = {'smurf_keepout':cut_smurfkeepout_indices,
            'freq_collisions':cut_f_indices,
            'squid_modulation':cut_fluxramp_indices,
            'lambda':cut_lamb_indices}


In [30]:
def cut_summary(df_wafer,cut_dict,N_possible):
    cut_channels = []
    for key in cut_dict.keys():
        cut_channels.extend(cut_dict[key])
    cut_channels = list(set(cut_channels))
    N_cut = len(cut_channels)
    
    N=len(df_wafer)
    resonator_yield = N/N_possible*100
    total_yield = ((N-N_cut) / N_possible)*100
    
    print('Total yield = %.1f%%.  %d cut of %d total'%(total_yield,N_cut,N_possible))
    print('Yield hits per category:\n')
    print('Resonator yield = %.1f%%.  %d resonators found out of %d possible.'%(resonator_yield,N,N_possible))
    for key in cut_dict.keys():
        N_cat = len(cut_dict[key])
        cat_yield = (1-N_cat / N)*100
        print(key, ' yield = %.1f%%.  %d cut of %d total'%(cat_yield,N_cat,N))
    
    # determine yield of SQUID-coupled readout channels
    res_only_indices = df_wafer[df_wafer["res_num"]==0].index.to_list()
    N_resonly = len(res_only_indices)
    N_squidchannels = N-N_resonly
    squid_channels_cut = cut_channels.copy()
    for dex in res_only_indices:
        if dex in squid_channels_cut:
            squid_channels_cut.remove(dex)
    N_squid_channels_cut = len(squid_channels_cut)
    N_possible_sq_chan = N_possible - N_resonly
    squid_channel_yield = (1-N_squid_channels_cut/N_possible_sq_chan)*100
    
    print('\nSQUID-coupled READOUT CHANNEL YIELD: %.1f%%. %d cut of %d possible.'%(squid_channel_yield,N_squid_channels_cut, N_possible_sq_chan))
    
    

In [31]:
cut_summary(df_wafer,cut_dict,66*N_chips)

Total yield = 68.0%.  279 cut of 924 total
Yield hits per category:

Resonator yield = 98.2%.  907 resonators found out of 924 possible.
smurf_keepout  yield = 92.5%.  68 cut of 907 total
freq_collisions  yield = 93.9%.  55 cut of 907 total
squid_modulation  yield = 89.4%.  96 cut of 907 total
lambda  yield = 87.8%.  111 cut of 907 total

SQUID-coupled READOUT CHANNEL YIELD: 69.9%. 274 cut of 910 possible.


In [32]:
chis_indices

[2842]

In [33]:
lamb_err_indices

[1771, 1776, 1779, 1785, 1786, 1799, 1816, 2299]

In [34]:
low_indices

[1802,
 1903,
 1919,
 1920,
 1927,
 1929,
 1930,
 1931,
 1932,
 1933,
 2038,
 2039,
 2040,
 2041,
 2048,
 2050,
 2054,
 2055,
 2056,
 2057,
 2058,
 2059,
 2060,
 2064,
 2089,
 2188,
 2197,
 2206,
 2221,
 2222,
 2294,
 2298,
 2307,
 2308,
 2322,
 2323,
 2331,
 2423,
 2424,
 2425,
 2427,
 2428,
 2429,
 2430,
 2431,
 2432,
 2433,
 2434,
 2435,
 2436,
 2437,
 2438,
 2439,
 2440,
 2441,
 2442,
 2443,
 2444,
 2445,
 2446,
 2447,
 2448,
 2449,
 2450,
 2451,
 2452,
 2453,
 2465,
 2472,
 2481,
 2591,
 2599,
 2608,
 2694,
 2832,
 2833,
 2834,
 2841,
 2856,
 2933,
 2965,
 3057,
 3070,
 3097,
 3101,
 3233,
 3335,
 3336,
 3453,
 3470,
 3484,
 3486,
 3493,
 3509]